In [ ]:
# Libraries

import numpy as np
from skimage.transform import resize
import pymartini
import rasterio
import pyvista as pv
from scipy.ndimage import map_coordinates
import settings
import utils

# matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib as matplot
# plotting
import seaborn as sns
# for interactions
from mpl_toolkits.mplot3d import Axes3D

import TIN_engine
from TIN_engine import *
from TIN_draw import *
from TIN_drainage import *
from TIN_watershed import *
from TIN import *

# remove this (and possibly restart the kernel) if you don't want interactive plots
%matplotlib widget

# for reloading modules (specifically TIN_engine) during development
%load_ext autoreload
%autoreload 2


# pysheds
from pysheds.grid import Grid

# other
import time
import copy

# Watershed Delineation With TIN

In [ ]:
# ---- Load DEM and create TIN ----
vertices_3d, triangles, xs, ys, zs, zs_scaled  = get_triangles_from_DEM(settings.WASHINGTON_SMALL)

In [ ]:
# ---- Get Highest Point ----
max_index = np.nanargmax(zs)
highest_point = [xs[max_index], ys[max_index], zs[max_index]]
highest_point

In [ ]:
# ---- Get Subset of Triangles Around Highest Point ----
radius = 0.02
bounds_around_highest = [highest_point[0] + radius, highest_point[1] + radius, highest_point[0] - radius, highest_point[1] - radius]

triangles_subset = get_subset_of_triangles_from_bounds(triangles, bounds_around_highest, xs, ys)

In [ ]:
# ---- Convert to Triangle and Vertex Objects ----
triangle_objects, vertices = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)
print(len(triangle_objects), "triangle objects created.")
print(len(vertices), "vertex objects created.")

In [ ]:
# ---- Preprocessing ----
has_flat_triangles(triangle_objects)
flat = get_flat_triangles(triangle_objects)
len(flat)
unflaten_triangles(triangle_objects)

In [ ]:
# ---- Drainage network calculation ----
drainage_outlet_nodes = create_drainage_network(triangle_objects)
print(len(drainage_outlet_nodes), "outlet nodes created.")

In [ ]:
# ---- Watershed delineation ----
watershed, start_node = delineate_watershed(drainage_outlet_nodes, highest_point)

print(len(watershed), "triangles")

In [ ]:
# ---- Visualize watershed ----

def draw_node(node: Node, depth, color):
    for upstream_node in node.upstream_nodes:
        draw_line_points(ax, node.point[0:2], upstream_node.point[0:2], color, linewidth=2)
        draw_node(upstream_node, depth + 1, color)

SHOW_DRAINAGE_NETWORK = True

# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing terrain triangles.")

for triangle in watershed:
    draw_triangle_object(ax, triangle, "#00ff0055", filled=True)

print("Done drawing watershed triangles.")

if SHOW_DRAINAGE_NETWORK:
    draw_point(ax, start_node.point[0:2], markersize=4)
    random_color = np.random.rand(3,)
    draw_node(start_node, 0, random_color)

plt.show()

# Watershed Delineation With Raster